In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import ExcelWriter
%matplotlib inline
import math
from datetime import datetime
import time

In [2]:
# import IDF
from AS import *
from feature_selection import *
from Binning import transform_df_to_woe, construction_binning
from AS_2 import *

In [40]:
# Перед названием пути к файлу надо поставить ../ чтобы подняться на папку выше.
df = pd.read_excel('../data_sql/ExcelFile_2021-12-20T17_30_46.xlsx')

df['date_requested'] = pd.to_datetime(df['date_requested'])
df_req = df[['credit_id', 'date_requested']]
y = df['target']
df['y'] = df['target']

In [41]:
# Filling missing values (np.nan, '_MISSING_')
cat_vars = df.select_dtypes(include=[object]).columns
df = filling(df)

min_num = 50
df, else_df = replace_not_frequent_2(df, cat_vars, num_min = min_num)

# Drop columns with only 1 value
df = drop_single_value_column(df)

In [42]:
df_out = df[df['date_requested'] >= '2021-05-01']
df = df[df['date_requested'] < '2021-05-01']

In [43]:
df_train, df_test, y_train, y_test = train_test_split(df, df.y, test_size=0.2, stratify=df.y, random_state=123)
y_out = df_out.y

In [44]:
iv_df = pd.read_excel('../results/iv_df_for_correction.xlsx', index_col=0).reset_index(drop=True)

In [17]:
iv_df[iv_df['VAR_NAME'] == '']

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,DR,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,WOE,IV
13,DTI_2,-0.044293,0.265415,580,0.125862,73,0.120861,507,0.077122,-0.449252,0.105996
14,DTI_2,0.265415,1.182705,4644,0.094100,437,0.723510,4207,0.639945,-0.122732,0.105996
15,DTI_2,1.182705,20.519541,1954,0.048106,94,0.155629,1860,0.282933,0.597733,0.105996


In [45]:
def RepresentsFloat(s):
    try: 
        float(s)
        return True
    except ValueError:
        return False

In [46]:
'''
ВАЖНО! NaN значения заоплняются так же, как это указано в изначальной
    excel таблице, так что их стоит менять там.
'''
iv_df_old = iv_df.copy()

In [47]:
# Пересчитываем WOE и IV для новых, исправленных ручным образом, бинов.
for col in iv_df['VAR_NAME'].unique():
# for col in ['SCORE_4_5', 'industry']:
    try:
        # Заменить df на требуемый DataFrame (df_train или другой)
        df1 = df_train[[col, 'target']].copy()
        iv_col = iv_df[iv_df['VAR_NAME'] == col]
    except:
        continue

    if RepresentsFloat(iv_col.iloc[0]['MIN_VALUE']):
        iv_df.loc[iv_df['VAR_NAME'] == col, 'MIN_VALUE'] = iv_df.loc[iv_df['VAR_NAME'] == col, 'MIN_VALUE'].astype(float)
        iv_df.loc[iv_df['VAR_NAME'] == col, 'MAX_VALUE'] = iv_df.loc[iv_df['VAR_NAME'] == col, 'MAX_VALUE'].astype(float)
        iv_col = iv_df[iv_df['VAR_NAME'] == col]

    for i, row in iv_col.iterrows():
        if pd.isna(row['MIN_VALUE']):
            val_count = df1[(df1[col].isna())]['target']
        elif type(row['MIN_VALUE']) in [int, float]:
        # elif RepresentsFloat(row['MIN_VALUE']):
            min_val, max_val = float(row['MIN_VALUE']), float(row['MAX_VALUE'])
            # TODO: поменять, чтобы был правильный учёт границ.
            val_count = df1[(df1[col] >= min_val) & (df1[col] <= max_val - 1e-8)]['target']
        else:
            categories = row['MIN_VALUE'].split(' | ')
            val_count = df1[df1[col].isin(categories)]['target']
        
        iv_df.loc[i, 'COUNT'] = val_count.count()
        iv_df.loc[i, 'EVENT'] = val_count.sum()

        iv_df.loc[i, 'NONEVENT'] = val_count.count() - val_count.sum()

    d3 = iv_df.loc[iv_df['VAR_NAME'] == col]
    iv_df.loc[iv_df['VAR_NAME'] == col, "EVENT_RATE"] = d3.EVENT / d3.EVENT.sum()
    iv_df.loc[iv_df['VAR_NAME'] == col, "NON_EVENT_RATE"] = d3.NONEVENT / d3.NONEVENT.sum()

    d3 = iv_df.loc[iv_df['VAR_NAME'] == col]
    iv_df.loc[iv_df['VAR_NAME'] == col, "WOE"] = np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)

    # Сохраняем тот резульатт для NaN, который предсатвлен в изначальном excel файле.
    iv_df.loc[(iv_df['VAR_NAME'] == col) & (iv_df['MIN_VALUE'].isna())] = iv_df_old[(iv_df['VAR_NAME'] == col) & (iv_df['MIN_VALUE'].isna())]

    iv_df.loc[iv_df['VAR_NAME'] == col, "IV"] = (d3.NON_EVENT_RATE-d3.EVENT_RATE)*np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)

    iv_df.loc[iv_df['VAR_NAME'] == col] = iv_df.loc[iv_df['VAR_NAME'] == col].replace([np.inf, -np.inf], 0)
    iv_df.loc[iv_df['VAR_NAME'] == col, 'IV'] = iv_df.loc[iv_df['VAR_NAME'] == col].IV.sum()

    if iv_df.loc[iv_df['VAR_NAME'] == col]['COUNT'].sum() != df1.shape[0]:
        print(f'ERROR columns "{col}" has less objects in bins than should')

In [ ]:
iv_df[iv_df['VAR_NAME'] == '']

In [49]:
iv_df.to_excel('../results/iv_df_after_correction.xlsx')